In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *

from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,KFold,GroupKFold,StratifiedGroupKFold,GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,StackingClassifier,BaggingClassifier
from xgboost import XGBClassifier
from sklearn.utils import shuffle

In [2]:
dataTrain=pd.read_csv("data/Train.txt",sep="#|;",engine="python",header=None,index_col = False)
dataTest=pd.read_csv("data/Test.txt",sep="#|;",engine="python",header=None,index_col = False)
dataCros=pd.read_csv("data/Cross_Validation.txt",sep="#|;",engine="python",header=None,index_col = False)

# data=shuffle(pd.concat([dataTrain,dataTest,dataCros]))
# data.reset_index(inplace=True)

In [8]:
dataCros.isnull().sum()

0      0
1      0
2      0
3      0
4      0
      ..
542    0
543    0
544    0
545    0
546    0
Length: 547, dtype: int64

In [280]:
data=data.drop("index",axis=1)

In [309]:
scaler=StandardScaler()
X=data.drop(0,axis=1)
X=scaler.fit_transform(X)
Y=data[0]
X_train,X_test,Y_train,Y_test,=train_test_split(X,Y,train_size=0.90)

IndexError: index 20188 is out of bounds for axis 0 with size 18170

In [284]:
%%time
def find_best_model(learning_algorithm,X,Y,expected_accuracy=0.75,model_selection=None,model_selection_params={},params={}):
    import pickle
    while True:
        train_size=np.round(np.random.random(500),2)
        train_size = train_size[(train_size>0.5)&(train_size<0.95)]
        for size in train_size:
            X_train,X_val,y_train,y_val = train_test_split(X,Y,train_size=size)
            
            #adding default parametrs If params is not empty and model_selection is not None
            if model_selection:
                for (key,value) in params.items():
                    model = learning_algorithm()
                    l=set()
                    if hasattr(value,"__iter__") and type(value)!=str:
                        l=set(value)
                    else:
                        l.add(value)
                    l.add(model.__dict__.get(key))
                    params[key]=list(l)
                selected_model = model_selection(model,params,scoring="accuracy",**model_selection_params)
                model = selected_model.fit(X_train,y_train)
            else:
                model = learning_algorithm(**params)
                model.fit(X_train,y_train)
            
            y_val_predict=model.predict(X_val)
            test_accuracy=accuracy_score(y_val_predict,y_val)

            print(f"Train Size: {size}, Model: {learning_algorithm.__name__}, Test Accuracy:{test_accuracy:.3}")
            if test_accuracy>=expected_accuracy:
                with open(f"{learning_algorithm.__name__.lower()}.pkl","wb") as file:
                    pickle.dump(model,file)
                return test_accuracy,model


CPU times: user 17 µs, sys: 0 ns, total: 17 µs
Wall time: 27.2 µs


In [212]:
accuracy,model=find_best_model(DecisionTreeClassifier,X_train,Y_train,expected_accuracy=0.77)

Train Size: 0.53, Model: DecisionTreeClassifier, Test Accuracy:0.742
Train Size: 0.55, Model: DecisionTreeClassifier, Test Accuracy:0.736
Train Size: 0.77, Model: DecisionTreeClassifier, Test Accuracy:0.77
Train Size: 0.7, Model: DecisionTreeClassifier, Test Accuracy:0.758
Train Size: 0.8, Model: DecisionTreeClassifier, Test Accuracy:0.776


In [221]:
y_predict=model.predict(X_test)
accuracy_score(y_predict,Y_test)

0.9742574257425742

In [215]:
param_log =   {
    'max_iter' : range(300,500,20)
    }
accuracy,model=find_best_model(LogisticRegression,X_train,Y_train,expected_accuracy=0.92,params={"max_iter":500})

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train Size: 0.6, Model: LogisticRegression, Test Accuracy:0.919
Train Size: 0.56, Model: LogisticRegression, Test Accuracy:0.922


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
print(DecisionTreeClassifier().__doc__)

A decision tree classifier.

    Read more in the :ref:`User Guide <tree>`.

    Parameters
    ----------
    criterion : {"gini", "entropy", "log_loss"}, default="gini"
        The function to measure the quality of a split. Supported criteria are
        "gini" for the Gini impurity and "log_loss" and "entropy" both for the
        Shannon information gain, see :ref:`tree_mathematical_formulation`.

    splitter : {"best", "random"}, default="best"
        The strategy used to choose the split at each node. Supported
        strategies are "best" to choose the best split and "random" to choose
        the best random split.

    max_depth : int, default=None
        The maximum depth of the tree. If None, then nodes are expanded until
        all leaves are pure or until all leaves contain less than
        min_samples_split samples.

    min_samples_split : int or float, default=2
        The minimum number of samples required to split an internal node:

        - If int, then cons

In [220]:
params={'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50],
 'max_features': ['log2', 'sqrt'],
 'min_samples_leaf': [2, 4],
 'min_samples_split': [ 5, 10],
 'n_estimators': [200, 400, 600],
 "criterion":["gini", "entropy", "log_loss"]
}

accuracy,model=find_best_model(RandomForestClassifier,X_train,Y_train,expected_accuracy=0.95)

Train Size: 0.94, Model: RandomForestClassifier, Test Accuracy:0.968


In [300]:
len(X_train)

18170

In [297]:
Y_xgb=Y_train[Y_train.isin(range(10))]
X_xgb=X_train[Y_xgb.index]

IndexError: index 18485 is out of bounds for axis 0 with size 18170

In [311]:
xgb_params = {"subsample":[0.5, 0.75, 1],
              "colsample_bytree":[0.5, 0.75, 1],
              "max_depth":[2, 6, 12],
              "min_child_weight":[1,5,15],
              "learning_rate":[0.3, 0.1, 0.03],
              "n_estimators":[120,150,200]}

D=data[data[0].isin(range(10))]
scaler=StandardScaler()
X=D.drop(0,axis=1)
X=scaler.fit_transform(X)
Y=D[0]
X_xgb_train,X_xgb_test,Y_xgb_train,Y_xgb_test=train_test_split(X,Y,train_size=0.90)
accuracy,model=find_best_model(XGBClassifier,X_xgb_train,Y_xgb_train,expected_accuracy=0.975)

Train Size: 0.83, Model: XGBClassifier, Test Accuracy:0.976


In [312]:
predict=model.predict(X_xgb_test)
accuracy_score(predict,Y_xgb_test)

0.9830316742081447

In [341]:
with open("xgbclassifier.pkl","rb") as file:
    model = pickle.load(file)
accuracy_score(model.predict(X_xgb_test),Y_xgb_test)

0.9830316742081447

In [ ]:
Y_predict=m.predict(X_test)
accuracy_score(Y_predict,Y_test)

In [10]:
DecisionTreeClassifier().__dict__

{'criterion': 'gini',
 'splitter': 'best',
 'max_depth': None,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'min_weight_fraction_leaf': 0.0,
 'max_features': None,
 'max_leaf_nodes': None,
 'random_state': None,
 'min_impurity_decrease': 0.0,
 'class_weight': None,
 'ccp_alpha': 0.0}

In [ ]:
DecisionTreeClassifier(**{})

In [ ]:
dir(())

In [ ]:
hasattr([],"__iter__")

In [ ]:
print(model.__dict__.get("estimator5"))

In [ ]:
DecisionTreeClassifier().__dict__

In [ ]:
print("sd")

In [25]:
a=set([1,2,3,4])
b=set([1,2,3,5,6,7])
a.union(b)

{1, 2, 3, 4, 5, 6, 7}

In [24]:
a

{4, 5, 6, 7}

In [1]:
import numpy as np

In [4]:
a=np.array([])

In [6]:
bool(a)

/tmp/ipykernel_3440/738525883.py:1: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  bool(a)


False